In [3]:
from clamv import *

import cv2
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

img = get_images()
X = img.data
y = img.target

imgc = get_images(color=True)
Xc = imgc.images
yc = imgc.target

# EigenFaces

In [2]:
test(OpenCVClassifier(cv2.face.createEigenFaceRecognizer), X, y)

start:  1480301985.2910101
end 1480303378.4732616 duration 1393.1822514533997


(0.59116617001688421, 0.048117681056166671)

In [4]:
test(TransformerClassifier(PCA(), LinearSVC()), X, y, "results/pca")

start:  1480447828.212188
end 1480448100.443965 duration 272.23177695274353


(0.58137634323460519, 0.04255781733909237)

In [4]:
pca = PCA(n_components=150, svd_solver='randomized', whiten=True)
param_grid = {
    'C': [1e3, 5e3, 1e4, 5e4, 1e5],
    'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
}
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), 
                   param_grid)
test(TransformerClassifier(pca, clf), X, y)

start:  1480431294.5438933
end 1480431687.8512075 duration 393.30731415748596


(0.86583376620658026, 0.036615743456890622)

In [4]:
test(ColorClassifier(PCA(), LinearSVC()), Xc, yc, 'color_eigen')

Wed Nov 30 22:08:57 2016
Fold Accuracy:0.674242424242
Fold Accuracy:0.746153846154
Fold Accuracy:0.646153846154
Fold Accuracy:0.623076923077
Fold Accuracy:0.651162790698
Fold Accuracy:0.635658914729
Fold Accuracy:0.6875
Fold Accuracy:0.700787401575
Fold Accuracy:0.700787401575
Fold Accuracy:0.706349206349
duration: 1719.7194814682007


(0.67718727545522095, 0.036233472024569641)

# FisherFaces

In [5]:
test(OpenCVClassifier(cv2.face.createFisherFaceRecognizer), X, y)

start:  1480431687.8591
end 1480433115.1510527 duration 1427.2919526100159


(0.72989531242006178, 0.056843754073469061)

In [4]:
test(TransformerClassifier(LinearDiscriminantAnalysis(), LinearSVC()),
     X, y, "results/fisher")

start:  1480453187.603773
duration: 84.91378283500671


(0.77392675354704399, 0.02764765728266802)

In [7]:
param_grid = {
    'C': [1e3, 5e3, 1e4, 5e4, 1e5],
    'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
}
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), 
                   param_grid)
test(TransformerClassifier(LinearDiscriminantAnalysis(), clf), X, y)

start:  1480433861.971472


/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.5/dist-packages/sklearn

end 1480434453.176393 duration 591.2049210071564


(0.71531755222259474, 0.052425966186629092)

In [8]:
param_grid = {
    'C': [1e3, 5e3, 1e4, 5e4, 1e5],
    'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
}
clf = GridSearchCV(SVC(kernel='linear', class_weight='balanced'), 
                   param_grid)
test(TransformerClassifier(LinearDiscriminantAnalysis(), clf), X, y)

start:  1480437319.8436885


/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.5/dist-packages/sklearn

end 1480437993.6637156 duration 673.8200271129608


(0.72612255896621747, 0.040414849929502825)

In [5]:
test(ColorClassifier(LinearDiscriminantAnalysis(), LinearSVC()),
     Xc, yc, 'color_fisher')

Wed Nov 30 22:37:37 2016
Fold Accuracy:0.840909090909
Fold Accuracy:0.815384615385
Fold Accuracy:0.846153846154
Fold Accuracy:0.830769230769
Fold Accuracy:0.806201550388
Fold Accuracy:0.84496124031
Fold Accuracy:0.7734375
Fold Accuracy:0.818897637795
Fold Accuracy:0.913385826772
Fold Accuracy:0.825396825397
duration: 1048.7258393764496


(0.83154973638782115, 0.034142800787140944)